In [28]:
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm
from joblib import Parallel, delayed

In [53]:
pc_transcripts = pd.read_csv("./data/protein_coding_transcripts.csv")

## Map (variant) position onto transcripts

In [45]:
def df_chunker(full_df, chunks):
    dfs = list()
    interval_size = full_df.shape[0]//chunks
    dfs.append(full_df.iloc[0:interval_size, :])

    for i in range(chunks - 1):
        dfs.append(full_df.iloc[(interval_size * (i + 1))
                   :(interval_size * (i + 2)), :])

    if full_df.shape[0] % chunks != 0:
        dfs.append(full_df.iloc[interval_size * chunks:, :])

    return dfs


# pri_table: table of ranges to which pos are assessed on
def in_range(pos, chrom, pri_table):
    sub_pri = pri_table[pri_table.seqname == chrom]
    starts = sub_pri["start"]
    ends = sub_pri["end"]
    
    in_ends = (ends - pos) >= 0
    in_starts = (pos - starts) >= 0
    
    between = (in_ends * in_starts)
    between = np.where(between)[0]
    if len(between) == 0:
        return None, None
    between = sub_pri.iloc[between]

    return between["gene_id"].item(), between["transcript_ids"].item()


def pos_transc_mapper(sub_df, pri_table):
    pos_tgs = sub_df[["CHR", "POS"]].apply(lambda x: in_range(x.POS, x.CHR, pri_table), axis=1)
    pos_gene_ids = [x[0] for x in pos_tgs]
    pos_tran_ids = [x[1] for x in pos_tgs]

    sub_df["GENE_ID"] = pos_gene_ids
    sub_df["TRANSCRIPT_ID"] = pos_tran_ids
    
    return sub_df

In [47]:
if __name__ == "__main__":
    start = time.time()
    samples = pd.read_csv('./data/long_read_samples_short.csv')
    # Protein coding only
    pc_pri = pd.read_csv("./data/pc_exon_region_classification2.csv")
    
    for subj_id, tissue_id in tqdm(zip(samples["SUBJECT_ID"][:2], samples["TISSUE_ID"][:2])):
        try:
            var_df = pd.read_csv(f"./GTEx_Analysis_v8_ASE_counts_by_subject/{subj_id}.v8.ase_table.tsv", sep="\t")
        except FileNotFoundError:
            continue

        sub_df = var_df[var_df.TISSUE_ID == tissue_id].copy()
        sub_df["COUNT"] = sub_df.ALT_COUNT + sub_df.REF_COUNT
        sub_df = sub_df[["CHR", "POS", "TISSUE_ID", "COUNT", "ALT_COUNT", "REF_COUNT"]]

        sdf_chunks = df_chunker(sub_df, 50)
        # valid_pri as second arg otherwise
        out_dfs = Parallel(n_jobs=-1)(delayed(pos_transc_mapper)(sdf_chunk, pc_pri) for sdf_chunk in sdf_chunks)

        # pd.concat(out_dfs, axis=0).to_csv(f"./data/shortReadMappedFilteredPCTransc/{subj_id}-{tissue_id}_variant_isospec.csv", index=False)
    print(time.time() - start)

2it [01:10, 35.34s/it]

71.40602922439575


In [65]:
if __name__ == "__main__":
    start = time.time()
    samples = pd.read_csv('./data/long_read_samples_short.csv')
    # Protein coding only
    pc_pri = pd.read_csv("./data/pc_exon_region_classification2.csv")

    # Protein coding transcripts
    valid_pri = valid_pri.merge(pc_transcripts)

    for subj_id, tissue_id in tqdm(zip(samples["SUBJECT_ID"], samples["TISSUE_ID"])):
        try:
            var_df = pd.read_csv(f"./GTEx_Analysis_v8_ASE_counts_by_subject/{subj_id}.v8.ase_table.tsv", sep="\t")
        except FileNotFoundError:
            continue
        
        # Tissue-specific filtering of transcript share
        filter_template = pd.read_csv(f'./data/SR_filtered_transcripts_by_tissue/{tissue_id}.csv')
        filtered_isospec = valid_pri.merge(filter_template[["gene_id", "transcript_id"]], on=["gene_id", "transcript_id"])

        sub_df = var_df[var_df.TISSUE_ID == tissue_id].copy()
        sub_df["COUNT"] = sub_df.ALT_COUNT + sub_df.REF_COUNT
        sub_df = sub_df[["CHR", "POS", "TISSUE_ID", "COUNT", "ALT_COUNT", "REF_COUNT"]]

        sdf_chunks = df_chunker(sub_df, 50)
        # valid_pri as second arg otherwise
        out_dfs = Parallel(n_jobs=-1)(delayed(pos_transc_mapper)(sdf_chunk, filtered_isospec) for sdf_chunk in sdf_chunks)

        pd.concat(out_dfs, axis=0).to_csv(f"./data/shortReadMappedShareFilteredPC/{subj_id}-{tissue_id}_variant_isospec.csv", index=False)
    print(time.time() - start)

88it [12:37,  8.61s/it]

758.3174850940704
